In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from kiteconnect import KiteConnect

# Initialise Kite
acctkn_file = r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\Brokers\acc_token.txt'
reqtkn_file = r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\Brokers\req_token.txt'
kite_access_token = open(acctkn_file,'r').read()
kite_request_token = open(reqtkn_file,'r').read()
print(kite_access_token)

# Load the data
df = pd.read_csv(r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\nf_backtest_consolidated.csv')
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%m/%d/%Y %H:%M')

df.set_index('DateTime', inplace=True)


In [ ]:
instruments_df = pd.read_csv(r'Brokers\instruments.csv')

In [12]:
# Empty list to store trades
trades = []

# Get unique dates from df
unique_dates = df.index.normalize().unique()
print("Unique dates:", unique_dates)

for i in range(1, len(unique_dates)):
    print(f"Processing date: {unique_dates[i]}")

    # Select data for the current day and the previous day
    previous_day = df[df.index.normalize() == unique_dates[i-1]]
    current_day = df[df.index.normalize() == unique_dates[i]]
    
    print("Previous day:", previous_day.head())

    # # Check if there is enough data for the previous day
    if previous_day.between_time('14:30', '15:15').empty:
        print("Not enough data for previous day.")
        continue
    
    # Calculate the 15 min weighted average and the trend
    previous_day_weighted_average = np.average(previous_day.between_time('14:30', '15:15')['Close'], 
                                               weights=range(1, len(previous_day.between_time('14:30', '15:15')) + 1))
    print(f"Weighted average for previous day: {previous_day_weighted_average}")

    trend = 'Long' if previous_day.at_time('15:15')['Close'].values[0] - previous_day_weighted_average > 0 else 'Short'
    print(f"Trend: {trend}")

    # Check if there is enough data for the current day
    if current_day.empty:
        print("No data for current day.")
        continue

    # Find the trade entry price at 3.16 pm on the previous day
    try:
        entry_price = previous_day.at_time('15:16')['Close'].values[0]
        print(f"Trade entry price: {entry_price}")
    except:
        print("Could not find trade entry price.")
        continue

    # Find the trade exit price at 9.18 am on the current day
    try:
        exit_price = current_day.at_time('9:18')['Close'].values[0]
        print(f"Trade exit price: {exit_price}")
    except:
        print("Could not find trade exit price.")
        continue

    # Calculate net points
    net_points = exit_price - entry_price if trend == 'Long' else entry_price - exit_price
    print(f"Net points: {net_points}")

    # Store the trade
    trades.append([trend, unique_dates[i].date(), '3:16 PM', entry_price, '9:18 AM', exit_price, net_points])

# Check if any trades were executed
if not trades:
    print("No trades executed.")




Unique dates: DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10',
               '2019-01-11', '2019-01-14',
               ...
               '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-15',
               '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19',
               '2023-05-22', '2023-05-23'],
              dtype='datetime64[ns]', name='DateTime', length=1068, freq=None)
Processing date: 2019-01-02 00:00:00
Previous day:                     Ticker      Open      High       Low     Close
DateTime                                                          
2019-01-01 09:08:00  NIFTY  10881.70  10881.70  10881.70  10881.70
2019-01-01 09:16:00  NIFTY  10884.10  10885.30  10872.30  10874.55
2019-01-01 09:17:00  NIFTY  10874.10  10879.15  10874.10  10874.90
2019-01-01 09:18:00  NIFTY  10873.85  10876.65  10873.85  10876.65
2019-01-01 09:19:00  NIFTY  10876.70  10876.70  10861.65  10863.

In [13]:
# Create a dataframe for the trades
trades_df = pd.DataFrame(trades, columns=['Trade_Type', 'Date', 'TradeEntryTime', 
                                          'TradeEntryPrice', 'TradeExitTime', 
                                          'TradeExitPrice', 'NetTradePoints'])

# Add Trade_No column
trades_df['Trade_No'] = range(1, len(trades_df) + 1)
trades_df = trades_df[['Trade_No', 'Trade_Type', 'Date', 'TradeEntryTime', 'TradeEntryPrice', 
                       'TradeExitTime', 'TradeExitPrice', 'NetTradePoints']]

# Save the results
trades_df.to_csv('backtest_results.csv', index=False)
